In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers.core import Dense,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import os
import cv2
import cv2 as ocv
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.preprocessing import image
from keras.utils import np_utils
from numpy.random import rand, shuffle
from keras.callbacks import ModelCheckpoint
from keras_preprocessing.image import ImageDataGenerator
from sklearn import preprocessing
from tensorflow.keras.callbacks import EarlyStopping
Data = []
Lables = []
classes=['Monkeypox','Others']
dir='/content/drive/MyDrive/Colab Notebooks/Fold1/Train'

for category in os.listdir(dir):
    newPath = os.path.join(dir,category)
    for img in os.listdir(newPath):
        img_path = os.path.join(newPath,img)
        if 'Thumbs.db' not in img_path:
            print(img_path)
            img=ocv.imread(img_path)
            #norm = (img - np.min(img)) / (np.max(img) - np.min(img))
            Data.append((np.array(ocv.resize(img,(100,100)))))
            Lables.append(classes.index(category))
combined = list(zip(Data,Lables))
shuffle(combined)
Data[:],Lables[:] = zip(*combined)
X_train = np.array(Data)
Y_train = np.array(Lables)
Y_train = np_utils.to_categorical(Y_train)

#Data Augmentation
dataGen = ImageDataGenerator(rotation_range=20,width_shift_range=0.01,height_shift_range=0.01,horizontal_flip=False,vertical_flip=False)
dataGen.fit(X_train)

resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet101(include_top=False,
                   input_shape=(100,100,3),
                   pooling='avg',classes=2,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)  
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(2, activation='softmax'))
resnet_model.summary()
resnet_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
early_stop = EarlyStopping(monitor = 'val_loss',mode='min',verbose = 1, patience = 5)

history=resnet_model.fit(X_train,Y_train, epochs=50,batch_size=32,validation_split=0.25,callbacks=[early_stop])

print(resnet_model.evaluate(X_train,Y_train))

fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

#Prediction
img = ocv.imread('/content/drive/MyDrive/Colab Notebooks/Fold1/Train/Monkeypox/P13_1_1.jpg')
img = np.array(img)
img = ocv.resize(img,(100,100))
img = img.reshape(1,100,100,3)
# print(np.shape(img))
print(classes[(np.argmax(resnet_model.predict(img)))])

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
